In [1]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms
from torch.autograd import Variable
import fashion_mnist

In [2]:
# Load fashion mnist data 
train_dataset =fashion_mnist.MNIST(root='fashion_mnist/', train=True, transform=transforms.ToTensor(), download=True)

In [3]:
test_dataset = dsets.MNIST(root='fashion_mnist/', 
                           train=False, 
                           transform=transforms.ToTensor())

In [4]:
# Hyperparameters 
batch_size = 100 
learning_rate = 0.0002
num_epoch = 10

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size = batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False
                                         )

In [6]:
class conv_net(nn.Module):
    def __init__(self):
            super().__init__()
            self.layer1 = nn.Sequential(
                nn.Conv2d(1, 16, kernel_size=3, padding=1),
                nn.BatchNorm2d(16),
                nn.ReLU(),
                nn.MaxPool2d((2)),
                nn.Dropout2d(0.25)
            )
            self.layer2 = nn.Sequential(
                nn.Conv2d(16, 32, kernel_size=3, padding=1),
                nn.BatchNorm2d(32),
                nn.ReLU(),
                nn.MaxPool2d((2)),
                nn.Dropout2d(0.25)

            )
            self.fc = nn.Linear(7*7*32, 10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        return self.fc(out)
        
cnn = conv_net()
cnn.cuda()

conv_net (
  (layer1): Sequential (
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Dropout2d (p=0.25)
  )
  (layer2): Sequential (
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Dropout2d (p=0.25)
  )
  (fc): Linear (1568 -> 10)
)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr = learning_rate)


In [8]:
best_loss = 1
loss = []
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        # optimize 
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epoch, i+1, len(train_dataset)//batch_size, loss.data[0]))
        if best_loss > loss.data[0]:
            print("best_loss => ", best_loss, "loss current => ", loss.data[0])
            best_loss = loss.data[0]
            torch.save(cnn, 'model.pkl')
            print('Epoch [%d/%d], Step[%d/%d], Loss:%.4f' % (epoch + 1, num_epoch + 1, i+1, len(train_dataset)
                                                //batch_size, loss.data[0]))

best_loss =>  1 loss current =>  0.9982818365097046
Epoch [1/11], Step[86/600], Loss:0.9983
best_loss =>  0.9982818365097046 loss current =>  0.9861401915550232


/home/rajat/pytorch_env/lib/python3.5/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type conv_net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch [1/11], Step[87/600], Loss:0.9861
best_loss =>  0.9861401915550232 loss current =>  0.8893821835517883
Epoch [1/11], Step[96/600], Loss:0.8894
Epoch [1/10], Iter [100/600] Loss: 0.8967
best_loss =>  0.8893821835517883 loss current =>  0.8555640578269958
Epoch [1/11], Step[115/600], Loss:0.8556
best_loss =>  0.8555640578269958 loss current =>  0.7987183928489685
Epoch [1/11], Step[124/600], Loss:0.7987
best_loss =>  0.7987183928489685 loss current =>  0.7956110239028931
Epoch [1/11], Step[138/600], Loss:0.7956
best_loss =>  0.7956110239028931 loss current =>  0.753864049911499
Epoch [1/11], Step[144/600], Loss:0.7539
best_loss =>  0.753864049911499 loss current =>  0.750221848487854
Epoch [1/11], Step[157/600], Loss:0.7502
best_loss =>  0.750221848487854 loss current =>  0.6883711218833923
Epoch [1/11], Step[159/600], Loss:0.6884
best_loss =>  0.6883711218833923 loss current =>  0.6389422416687012
Epoch [1/11], Step[167/600], Loss:0.6389
best_loss =>  0.6389422416687012 loss curre

In [9]:
cnn.eval()    # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
cnn = torch.load('model.pkl')
for images, labels in test_loader:
    images = Variable(images).cuda()
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

In [10]:
print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))



Test Accuracy of the model on the 10000 test images: 87 %


In [11]:
class VGG(nn.Module):
    def __init__(self):
        super.__init__()
        
        layer1 = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d((2))
        )
        layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d((2))
        )
        nn.Linear()

In [ ]:
cnn.